<a href="https://colab.research.google.com/github/komekome4869/Machine-Learning/blob/main/%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB%E5%85%A5%E9%96%80/ch01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! cat /etc/os-release | head -n 2

PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"


In [5]:
!pip install transformers[ja,sentencepiece,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-n

transformersにはモデルを簡単に扱うためのpipelinesという機能が存在

In [6]:
from transformers import pipeline

## 第1章 はじめに

### 1.1 transformersを使って自然言語処理を解いてみよう

#### 1.1.1 文書分類

In [10]:
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)
positive_text = "世界には言葉がわからなくても感動する音楽がある。"
# positive_textの極性を予測
print(text_classification_pipeline(positive_text)[0])

{'label': 'positive', 'score': 0.9993619322776794}


In [17]:
# negative_testの極性を予測
negative_text = "世界には言葉が出ないほどひどい音楽がある。"
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9859846830368042}


#### 1.1.2 自然言語推論(natural language inference; NLI)

In [18]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text="研究室の５人が集まって会議を行なっています。"
entailment_text = "研究室で会議を行なっている人が5人います。"

#textとentailment_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair": entailment_text}))

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

{'label': 'entailment', 'score': 0.9980722665786743}


In [19]:
contradiction_text = "5人の研究室のメンバーがコンビニに行っています。"
#textとcontradiction_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair":contradiction_text}))

{'label': 'contradiction', 'score': 0.8066152334213257}


In [21]:
neutral_text = "5人の研究室のメンバーが座っています。"
#textとneutral_textの論理関係を予測
print(nli_pipeline({"text":text, "text_pair":neutral_text}))

{'label': 'neutral', 'score': 0.9876216053962708}


"entailment"とは「含意」を意味し、上記の例では「二人の男性がジェット機を見ています」
が成立するならば、「ジェット機を見ている人が二人います」も成立するという関係を表して
います。"contradiction"は「矛盾」であり、「二人の男性がジェット機を見ています」とい
う状況は、「二人の男性が飛んでいます」と矛盾しているため、このラベルが予測されます。
"neutral"は「中立」であり、「含意」とも「矛盾」とも判断がつかないテキストのペアに与え
られるラベルです。

#### 1.1.3 意味的類似度計算(semantic textual similatity; STS)

意味的類似度計算のモデルllm-book/bert-base-japanese-v3-jstsは２つのテキストの意味的類似度を0から5の範囲で予測する．

In [22]:
text_sim_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jsts",
                              function_to_apply="none",)
text = "研究室には機械学習を専門とする学生がいます。"
sim_text = "AIに関して勉強している生徒が研究室に所属している。"
#textとsim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

config.json:   0%|          | 0.00/796 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

3.2781925201416016


In [23]:
dissim_text="研究室にはお菓子がたくさんある。"
#textとdissim¥_textの類似度の計算
result = text_sim_pipeline({"text":text, "text_pair":dissim_text})
print(result["score"])

0.7104209065437317


文埋め込みを使ったベクトルのコサイン類似度の計算

In [24]:
from torch.nn.functional import cosine_similarity
sim_enc_pipeline = pipeline(
  model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
  task="feature-extraction",
)
# text と sim_text のベクトルを獲得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
# text と sim_text の類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

0.7652466893196106


In [25]:
#dissim_textのベクトルの獲得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
#textとdissim_textの類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.5930401086807251


#### 1.1.4 固有表現認識(named entity recognition; NER)

In [26]:
from pprint import pprint
ner_pipeline = pipeline(
  model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
  aggregation_strategy="simple",
)
text = "大谷翔平は岩手県水沢市出身のプロ野球選手"
# text 中の固有表現を抽出
pprint(ner_pipeline(text))

config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'end': None,
  'entity_group': '人名',
  'score': 0.99823624,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9986874,
  'start': None,
  'word': '岩手 県 水沢 市'}]


出力の"word"は抽出した固有表現の語句、"entity_group"は固有表現の種類、"score"は
ラベルの予測スコア、"start"と"end"は固有表現の開始位置と終了位置を示しています。
"start"と"end"は、本書執筆時の transformers の実装の問題9で、None が出力されていま
す。開始位置、終了位置を含めて正しく検出できるコードは第 6 章で示します。

#### 1.1.5 要約生成(summarization generation)

In [27]:
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)
article="ChatGPT をはじめとする大規模言語モデルが世界的に大きな注目を集めています。大規模言語モデルは、大規模なテキストデータを使って訓練された大規模なパラメータで構成されるニューラルネットワークです。自然言語処理や機械学習の研究で培われてきた多くの知見をもとに開発され、特に 2020 年以降にパラメータ数およびテキストデータを大規模化することで飛躍的に性能が向上しました。近年の大規模言語モデルは非常に高性能であり、コンピュータが我々の住む世界のことをよく理解し、人間と自然にやりとりできるような新しい能力を獲得したと言えるでしょう。また、近年はライブラリの充実や計算環境の普及にともなって、プログラミングの経験が少しあれば、ニューラルネットワークのモデルを手軽に動かしたり、データから学習させたりすることができるようになりました。大規模言語モデルの開発では、Hugging Face が開発する transformers という Python ライブラリが標準的に利用されています。このライブラリは、非常に洗練されたインターフェイスを提供しており、数十行程度のコードを書くだけで高度な処理を実装できます。本書は大規模言語モデルの技術的な側面に興味を持つ方に向けた入門書です。執筆にあたっては、理論と実装(プログラミング)の双方をバランスよく学べることを重視しました。本書の前半の理論的な解説では、大規模言語モデルの “先祖” である word2vec から ChatGPT に至るまでの技術的な変遷を系統立てて解説することを目指しました。またモデルの説明には、数式だけでなく図や例を加えて、なるべく直感的に理解できるように工夫しました。本書の後半では、代表的な自然言語処理タスクについて transformersなどを使って手を動かしながら、自然言語処理モデルの開発を学びます。本書ではすべて日本語のデータセットを使っており、実用的に使える日本語自然言語処理のモデルを作成します。本書の内容を一通り読むことで、大規模言語モデルのしくみを理解するとともに、自然言語処理のさまざまな課題を解決するモデルを開発できる基礎的な知識が身に付くはずです。大規模言語モデルは新しい技術であり、実世界の問題をどのように解決できるのかはまだ未知数です。しかし、今後は特定の用途に特化したモデルが多数開発されていき、実用化における個別の課題を乗り越えて、非常に幅広い問題に応用されていくことは間違いないように思われます。こうした中で、本書は、現場で大規模言語モデルの開発や研究に携わる人に役立つ日本語のまとまった書籍を作りたいという想いから始まりました。本書をきっかけに、大規模言語モデルに携わる人が増えることで、日本国内における研究や開発、ひいては産業の活性化につながることを願っています。"
print(text2text_pipeline(article)[0]["generated_text"])

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


大規模言語モデル入門 入門書【売れ筋チェック】


### 1.2 transformersの基本的な使い方

In [28]:
from transformers import AutoTokenizer
#AutoTokenizerでトークナイザをロードする
tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")
#入力文をトークンに分割する
tokenizer.tokenize("今日は天気が良いので")

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


['▁', '今日', 'は', '天気', 'が良い', 'の', 'で']

In [29]:
from transformers import AutoModelForCausalLM

#生成を行うモデルであるAutoModelForCausalLMを使ってモデルをロードする
model = AutoModelForCausalLM.from_pretrained("abeja/gpt2-large-japanese")
#トークナイザを使ってモデルへの入力を作成する
inputs = tokenizer("今日は天気が良いので", return_tensors="pt")
#後続のテキストを予測する
outputs = model.generate(
    **inputs,
    max_length=15,  #紫衣性する最大のトークン数
    pad_token_id=tokenizer.pad_token_id #パディングのトークンIDを指定
)
#generate関数の出力をテキストに変換する
generated_text=tokenizer.decode(
    outputs[0], skip_special_tokens=True
)
print(generated_text)

config.json:   0%|          | 0.00/974 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

今日は天気が良いので外でお弁当を食べました。
